In [2]:
print("Hi! from Agents")

Hi! from Agents


In [3]:
from file_conversion import FileConverter
from crewai import Agent,Crew,Task, LLM
from crewai_tools import FileReadTool,ScrapeWebsiteTool,MDXSearchTool,SerperDevTool
import os
from dotenv import load_dotenv

c:\Users\amish\anaconda3\envs\multi_agent\lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [4]:
converter_instance  = FileConverter()
converter_instance.pdf_to_md(r"C:\Users\amish\Downloads\RESUME.pdf")

Conversion done! See old_resume.md


In [5]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [6]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./old_resume.md')
search_resume = MDXSearchTool(mdx='./old_resume.md')

c:\Users\amish\anaconda3\envs\multi_agent\lib\site-packages\alembic\config.py:577: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


In [ ]:
# 1st Agent: Researcher
first_agent_llm = LLM(
    model=os.getenv("GEMINI_MODEL_NAME"),
    temperature=0.7,
)

researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=first_agent_llm,
    allow_delegation=True
)

# 2nd Agent: Profiler(OpenAI LLM)
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    allow_delegation= True
)

# 3rd Agent: Resume Strategist
third_agent_llm = LLM(
    model=os.getenv("OLLAMA_LLM_NAME"),
    base_url =os.getenv("OLLAMA_LOCAL_HOST"),
)
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=third_agent_llm,
    allow_delegation = True
)

# 4th Agent: Interview Preparer

interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    allow_delegation=True
)

In [8]:
# 1st Agent Task: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

# 2nd Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

# 3rd Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="new_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

# 4th Agent: Design Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

In [9]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [10]:
job_application_inputs = {
    'job_posting_url': 'https://careers.capgemini.com/job/Pune-GenAI-Engineer/1198919201/?feedId=388933&utm_source=LinkedInJobPostings',
    'github_url': 'https://github.com/amishkr22',
    'personal_writeup': """I am Amish Nayar, a passionate and driven Computer Science and Engineering undergraduate with a strong
                           focus on Machine Learning, Natural Language Processing, and Large Language Models. My hands-on
                           experience ranges from developing sophisticated AI-powered chatbots and RAG pipelines to fine-tuning
                           cutting-edge models like LLaMA and GEMMA."""
}

In [11]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e0aeb464-3ba9-442b-b982-b8b8dc854c49                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\amish\anaconda3\envs\multi_agent\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Job Researcher                                                                                     │
│                                                                                                                 │
│  Task: Analyze the job posting URL provided                                                                     │
│  (https://careers.capgemini.com/job/Pune-GenAI-Engineer/1198919201/?feedId=388933&utm_source=LinkedInJobPostin  │
│  gs) to extract key skills, experiences, and qualifications required. Use the tools to gather content and       │
│  identify and categorize the requirements.                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Personal Profiler for Engineers                                                                         │
│                                                                                                                 │
│  Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/amishkr22)     │
│  URLs, and personal write-up (I am Amish Nayar, a passionate and driven Computer Science and Engineering        │
│  undergraduate with a strong                                                                                    │
│                             focus on Machine Learning, Natural Language Processing, and Large Language Models.  │
│  My hands-on                                                                                                    │
│                             experience ranges from developing sophisticated AI-powered chatbots and RAG         │
│  pipelines to fine-tuning                                                                                       │
│                             cutting-edge models like LLaMA and GEMMA.). Utilize tools to extract and            │
│  synthesize information from these sources.                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\amish\anaconda3\envs\multi_agent\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {                                   │
│    "error": {                                                                                                   │
│      "code": 429,                                                                                               │
│      "message": "You exceeded your current quota, please check your plan and billing details. For more          │
│  information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",                       │
│      "status": "RESOURCE_EXHAUSTED",                                                                            │
│      "details": [                                                                                               │
│        {                                                                                                        │
│          "@type": "type.googleapis.com/google.rpc.QuotaFailure",                                                │
│          "violations": [                                                                                        │
│            {                                                                                                    │
│              "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",            │
│              "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",                                 │
│              "quotaDimensions": {                                                                               │
│                "model": "gemini-2.0-flash",                                                                     │
│                "location": "global"                                                                             │
│              },                                                                                                 │
│              "quotaValue": "15"                                                                                 │
│            }                                                                                                    │
│          ]                                                                                                      │
│        },                                                                                                       │
│        {                                                                                                        │
│          "@type": "type.googleapis.com/google.rpc.Help",                                                        │
│          "links": [                                                                                             │
│            {                                                                                                    │
│              "description": "Learn more about Gemini API quotas",                                               │
│              "url": "https://ai.google.dev/gemini-api/docs/rate-limits"                                         │
│            }                                                                                                    │
│          ]                                                                                                      │
│        },                                                                                                       │
│        {                                                                                                        │
│          "@type": "type.googleapis.com/google.rpc.Retry


 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 80e7ea4d-8515-49f6-9b45-7461827b11ca                                                                     │
│  Agent: Resume Strategist for Engineers                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: e0aeb464-3ba9-442b-b982-b8b8dc854c49                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "model": "gemini-2.0-flash",
              "location": "global"
            },
            "quotaValue": "15"
          }
        ]
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "description": "Learn more about Gemini API quotas",
     